In [1]:
import sqlite3
import time
import statistics

In [2]:
db_path = 'Dirigo.sqlite'

query = '''
WITH FirstPickup AS (
    SELECT MIN(e.timestamp) AS PickupTimestamp
    FROM OCEL_E2O er
    JOIN OCEL_Events e ON er.Event_id = e.Event_id
    WHERE e.Activity = 'Lodge Pickup Plan'
      AND er.object_id = 'Cr1'
)
SELECT "Cargo stock weight(scheduled)"
FROM OCEL_Cargo
WHERE object_id = 'Cr1'
  AND timestamp < (SELECT PickupTimestamp FROM FirstPickup)
  AND timestamp = (
    SELECT MAX(timestamp)
    FROM OCEL_Cargo
    WHERE object_id = 'Cr1'
      AND timestamp < (SELECT PickupTimestamp FROM FirstPickup)
);
'''
repeat = 1000

conn = sqlite3.connect(db_path)
cursor = conn.cursor()


durations = []

for i in range(repeat):
    start_time = time.time()
    cursor.execute(query)
    cursor.fetchall()  
    end_time = time.time()
    
    duration = (end_time - start_time) * 1000  
    durations.append(duration)
    # print(f"Run {i+1}: {duration:.4f} ms")


avg = statistics.mean(durations)
std_dev = statistics.stdev(durations)

print("\n=== Execution Statistics ===")
print(f"Average execution time: {avg:.4f} ms")
print(f"Standard deviation: {std_dev:.4f} ms")


cursor.close()
conn.close()


=== Execution Statistics ===
Average execution time: 0.1740 ms
Standard deviation: 0.0431 ms
